In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
import kagglehub

path = kagglehub.dataset_download("abdallahalidev/plantvillage-dataset")
print("Path to dataset files:", path)


c:\Users\preme\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.04G/2.04G [05:33<00:00, 6.57MB/s] 

Extracting files...


Path to dataset files: C:\Users\preme\.cache\kagglehub\datasets\abdallahalidev\plantvillage-dataset\versions\3


In [8]:
!ls /kaggle/input/plantvillage-dataset

!ls "/kaggle/input/plantvillage-dataset/plantvillage dataset"


'ls' is not recognized as an internal or external command,
operable program or batch file.
'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import os

# This is where your dataset is
dataset_path = "/kaggle/input/plantvillage-dataset/plantvillage dataset"

# Go specifically to the color images
color_path = os.path.join(dataset_path, "color")
print("Using dataset from:", color_path)



Using dataset from: /kaggle/input/plantvillage-dataset/plantvillage dataset\color


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/plantvillage-dataset/plantvillage dataset\\color'

In [5]:
import os

os.makedirs('binary_dataset/Healthy', exist_ok=True)
os.makedirs('binary_dataset/Not_Healthy', exist_ok=True)


In [6]:
dataset_path = "/kaggle/input/plantvillage-dataset/plantvillage dataset"
# Go specifically to the color images
color_path = os.path.join(dataset_path, "color")
print("Using dataset from:", color_path)


Using dataset from: /kaggle/input/plantvillage-dataset/plantvillage dataset\color


In [7]:
import shutil

source_dir = '/kaggle/input/plantvillage-dataset/color'
dest_dir = 'binary_dataset'

for folder in os.listdir(source_dir):
    folder_path = os.path.join(source_dir, folder)
    if os.path.isdir(folder_path):
        # Determine destination
        if 'healthy' in folder.lower():
            target_folder = os.path.join(dest_dir, 'Healthy')
        else:
            target_folder = os.path.join(dest_dir, 'Not_Healthy')
        
        # Copy all images
        for img_file in os.listdir(folder_path):
            src_img = os.path.join(folder_path, img_file)
            dst_img = os.path.join(target_folder, img_file)
            shutil.copy(src_img, dst_img)


FileNotFoundError: [WinError 3] The system cannot find the path specified: '/kaggle/input/plantvillage-dataset/color'

In [9]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = 'binary_dataset'

# Image transforms (resize + normalization)
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])  # ImageNet stats
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(data_dir), transform=transform)

# Split train/val manually (80/20)
from torch.utils.data import random_split

total_len = len(train_dataset)
train_len = int(0.8 * total_len)
val_len = total_len - train_len
train_dataset, val_dataset = random_split(train_dataset, [train_len, val_len])

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

FileNotFoundError: Found no valid file for the classes Healthy, Not_Healthy. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pretrained EfficientNetB0
model = models.efficientnet_b0(weights='IMAGENET1K_V1')
# Replace classifier for binary output
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)  # 1 output
model = model.to(device)

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\preme/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:02<00:00, 9.88MB/s]


In [11]:
criterion = nn.BCEWithLogitsLoss()  # combines sigmoid + BCE
optimizer = optim.Adam(model.parameters(), lr=1e-3)
from tqdm import tqdm

epochs = 5  # increase later
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.float().unsqueeze(1).to(device)  # BCE expects float
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = torch.sigmoid(outputs) > 0.5
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {train_acc:.4f}")

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            preds = torch.sigmoid(outputs) > 0.5
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)
    val_acc = val_correct / val_total
    print(f"Validation Accuracy: {val_acc:.4f}")

NameError: name 'train_loader' is not defined

In [13]:
from PIL import Image
from torchvision.transforms import ToTensor

img_path = 'image.png'
img = Image.open(img_path).convert('RGB')
img = transform(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    output = model(img)
    pred = torch.sigmoid(output) > 0.5
    print("Prediction:", "Not Healthy" if pred.item() else "Healthy")

Prediction: Not Healthy


In [17]:
torch.save(model.state_dict(), 'model_weights.pth')
print("Model weights saved to 'model_weights.pth'")


Model weights saved to 'model_weights.pth'


In [12]:
model = models.efficientnet_b0(weights='IMAGENET1K_V1')
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 1)
model.to(device)

model.load_state_dict(torch.load('model_weights.pth', map_location=device))
model.eval()  # Set to evaluation mode
print("Model weights loaded!")


Model weights loaded!
